In [1]:
import pandas
import json

dataFrame = pandas.read_excel("bd.xlsx", skiprows = range(0, 4)) # Base de dados
dF = dataFrame[["Instituição", "UF", "Conta", "Valor"]] # Base de dados com as colunas que iremos utilizar

# Áreas que iremos somar as despesas
areas_de_interesse = (dF["Conta"].str.startswith("10")) | (dF["Conta"] == "06 - Segurança Pública") | (dF["Conta"] == "08 - Assistência Social") | (dF["Conta"] == "09 - Previdência Social") | (dF["Conta"] == "12 - Educação")

despesas_selecionadas = dF.loc[areas_de_interesse] # Base de dados com as áreas filtradas
# despesas_selecionadas.loc[:, "Valor"] = pandas.to_numeric(despesas_selecionadas["Valor"], errors="coerce") # Transforma em NaN as células vazias
despesas_selecionadas.loc[:, "Valor"] = pandas.to_numeric(despesas_selecionadas["Valor"], errors="coerce").fillna(0)

# Soma os valores das despesas de cada área por estado e por município
despesas_por_estado = despesas_selecionadas[["UF", "Conta", "Valor"]].groupby(["UF", "Conta"]).sum().reset_index()
despesas_por_municipio = despesas_selecionadas[["Instituição", "Conta", "Valor"]].groupby(["Instituição", "Conta"]).sum().reset_index()

with pandas.ExcelWriter('TabelaFinal.xlsx') as writer: #Plota a tabela com duas abas, uma dos estados e uma dos municípios
    despesas_por_municipio.to_excel(writer, sheet_name='Por-Município', index=False)
    despesas_por_estado.to_excel(writer, sheet_name='Por-Estado', index=False)

despesas_por_estado.to_json(r'./frontend/src/data/table_Data.json')
despesas_por_municipio.to_json(r'./frontend/src/data/tabelData.json')

display(despesas_por_estado)
display(despesas_por_municipio)


dict = {}
for index, row in despesas_selecionadas.iterrows():
    uf = row['UF']
    instituicao = row['Instituição']
    conta = row['Conta']
    valor = row['Valor']

    if uf not in dict:
        dict[uf] = {}
    
    if instituicao not in dict[uf]:
        dict[uf][instituicao] = {}
    
    dict[uf][instituicao][conta] = valor

json.dumps(dict, indent = 2) 
with open("./frontend/src/data/tableDataMun.json", "w") as outfile:
    json.dump(dict, outfile)
print(dict)






,UF,Conta,Valor
0,AC,06 - Segurança Pública,8396592.59
1,AC,08 - Assistência Social,153330804.32
2,AC,09 - Previdência Social,124981218.1
3,AC,10 - Saúde,949692081.92
4,AC,10.122 - Administração Geral,4865381.04
...,...,...,...
303,TO,10.303 - Suporte Profilático e Terapêutico,51983408.44
304,TO,10.304 - Vigilância Sanitária,39117644.78
305,TO,10.305 - Vigilância Epidemiológica,119938769.17
306,TO,10.306 - Alimentação e Nutrição,3921568.21


,Instituição,Conta,Valor
0,Prefeitura Municipal de Abadia de Goiás - GO,06 - Segurança Pública,242147.46
1,Prefeitura Municipal de Abadia de Goiás - GO,08 - Assistência Social,3793058.61
2,Prefeitura Municipal de Abadia de Goiás - GO,09 - Previdência Social,4615720.65
3,Prefeitura Municipal de Abadia de Goiás - GO,10 - Saúde,18854955.69
4,Prefeitura Municipal de Abadia de Goiás - GO,10.301 - Atenção Básica,2463087.96
...,...,...,...
46076,Prefeitura Municipal de Óleo - SP,08 - Assistência Social,1900941.84
46077,Prefeitura Municipal de Óleo - SP,09 - Previdência Social,3050842.35
46078,Prefeitura Municipal de Óleo - SP,10 - Saúde,11869534.11
46079,Prefeitura Municipal de Óleo - SP,10.301 - Atenção Básica,11869534.11


{'RS': {'Prefeitura Municipal de Barra Funda - RS': {'06 - Segurança Pública': 92729.38, '08 - Assistência Social': 540583.13, '09 - Previdência Social': 1175658.48, '10 - Saúde': 3825025.34, '10.301 - Atenção Básica': 3744185.6, '10.302 - Assistência Hospitalar e Ambulatorial': 20569.96, '10.304 - Vigilância Sanitária': 23855.87, '10.305 - Vigilância Epidemiológica': 36413.91, '12 - Educação': 3416360.25}, 'Prefeitura Municipal de São José do Hortêncio - RS': {'06 - Segurança Pública': 90000.0, '08 - Assistência Social': 2027.51, '09 - Previdência Social': 4975.0, '10 - Saúde': 79793.7, '10.301 - Atenção Básica': 79793.7, '10.302 - Assistência Hospitalar e Ambulatorial': 1337.94, '12 - Educação': 28825.73}, 'Prefeitura Municipal de Três Cachoeiras - RS': {'06 - Segurança Pública': 1353.25, '08 - Assistência Social': 6772.85, '10 - Saúde': 35205.33, '10.301 - Atenção Básica': 34870.92, '10.305 - Vigilância Epidemiológica': 334.41, '12 - Educação': 105624.63}, 'Prefeitura Municipal de S